# Validation results

In this notebook the validation results are aggregated and evaluated

### imports

In [ ]:
import os
import pandas as pd
os.getcwd()

## Analysis

In [2]:
folders = [f.path for f in os.scandir('./ball-tracking/') if f.is_dir() and "train" in str(f.path) ]
folders

['./ball-tracking/train_yolov10n',
 './ball-tracking/train_yolov9s',
 './ball-tracking/train_yolov10s',
 './ball-tracking/train_yolov10m',
 './ball-tracking/train_yolov8s',
 './ball-tracking/train_yolov5su',
 './ball-tracking/train_yolov10l',
 './ball-tracking/train_yolo11s',
 './ball-tracking/train_yolov10x']

In [3]:
def process_yolo_results(folder_path):

    # open results.csv in folder
    results = pd.read_csv(os.path.join(folder_path, "results.csv"))
    
    # add a column with folder name
    results['model'] = folder_path.split("/")[-1]

    return results

In [4]:
for folder in folders:
    results = process_yolo_results(folder)
    if folder == folders[0]:
        all_results = results
    else:
        all_results = pd.concat([all_results, results])

all_results.head()

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2,model
0,1,45.8155,6.15668,19.61550,2.00021,0.66779,0.34581,0.39468,0.17823,4.50381,4.93404,1.72441,0.003317,0.003317,0.003317,train_yolov10n
1,2,83.0463,4.94959,5.36315,1.78710,0.59498,0.36452,0.40174,0.15001,4.89309,4.97917,1.76749,0.006584,0.006584,0.006584,train_yolov10n
2,3,120.3060,4.88140,4.34683,1.79225,0.20334,0.23515,0.15294,0.05110,5.50590,8.62437,1.84830,0.009786,0.009786,0.009786,train_yolov10n
3,4,157.3840,4.66396,4.04411,1.78109,0.36590,0.18959,0.11976,0.02130,8.17903,5.28695,2.09792,0.009703,0.009703,0.009703,train_yolov10n
4,5,194.5810,4.26424,3.61572,1.74288,0.54894,0.45037,0.44516,0.22211,4.06226,4.31499,1.69001,0.009604,0.009604,0.009604,train_yolov10n


In [5]:
all_results.groupby('model')['epoch'].max().reset_index()

,model,epoch
0,train_yolo11s,71
1,train_yolov10l,65
2,train_yolov10m,64
3,train_yolov10n,72
4,train_yolov10s,63
5,train_yolov10x,61
6,train_yolov5su,69
7,train_yolov8s,56
8,train_yolov9s,97


In [6]:
# group by model and get best model based on map50
best_results = all_results.groupby('model')['metrics/mAP50-95(B)'].max().reset_index()

In [8]:
best_models = pd.merge(all_results, best_results, on=['model', 'metrics/mAP50-95(B)'], how='inner')

best_models['f1'] = 2 * (best_models['metrics/precision(B)'] * best_models['metrics/recall(B)']) / (best_models['metrics/precision(B)'] + best_models['metrics/recall(B)'])

# create a column for time per epoch
best_models['time_epoch'] = best_models['time'] / best_models['epoch']

# define relevant columns
columns=['model', 'epoch', 'metrics/mAP50(B)','f1', 'metrics/precision(B)', 'metrics/recall(B)', 'time_epoch']

# return only those columns
best_models[columns]

# return only those columns
best_models[columns]

,model,epoch,metrics/mAP50(B),f1,metrics/precision(B),metrics/recall(B),time_epoch
0,train_yolov10n,66,0.72297,0.739100,0.90098,0.62653,37.223788
1,train_yolov9s,89,0.78365,0.792658,0.90247,0.70667,64.274494
2,train_yolov10s,55,0.77653,0.783806,0.91068,0.68796,57.764545
3,train_yolov10m,56,0.78425,0.786263,0.89717,0.69976,95.564464
4,train_yolov8s,48,0.76392,0.761371,0.91102,0.65395,45.416042
5,train_yolov5su,61,0.78854,0.789991,0.90558,0.70057,44.570820
6,train_yolov10l,57,0.81064,0.797161,0.88713,0.72376,140.211228
7,train_yolo11s,63,0.77968,0.780791,0.92602,0.67494,47.529365
8,train_yolov10x,53,0.81223,0.805263,0.89804,0.72986,195.260377
